In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
import pinecone
from pinecone import Pinecone, ServerlessSpec

In [6]:
pinecone = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pinecone.create_index(
    name="rag",dimension=1536, metric="cosine",spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [9]:
import json
file = open("reviews.json", "r")
data=json.load(file)
data["reviews"]


[{'id': 1,
  'professorName': 'Dr. Smith',
  'course': 'Introduction to Psychology',
  'rating': 4.5,
  'date': '2024-03-15',
  'comment': 'Dr. Smith is an excellent lecturer. The course material was challenging but presented in an engaging way.'},
 {'id': 2,
  'professorName': 'Prof. Johnson',
  'course': 'Advanced Calculus',
  'rating': 3.8,
  'date': '2024-02-28',
  'comment': 'Prof. Johnson knows the subject well, but sometimes struggles to explain complex concepts clearly.'},
 {'id': 3,
  'professorName': 'Dr. Lee',
  'course': 'World History',
  'rating': 4.9,
  'date': '2024-04-02',
  'comment': "Dr. Lee's passion for history is contagious. The class discussions were always thought-provoking."},
 {'id': 4,
  'professorName': 'Prof. Garcia',
  'course': 'Organic Chemistry',
  'rating': 3.2,
  'date': '2024-03-20',
  'comment': "The course was very difficult, and Prof. Garcia's teaching style didn't help much. More office hours would be appreciated."},
 {'id': 5,
  'professorName'

In [16]:
processed_data=[]
client=OpenAI()

for review in data["reviews"]:
    response = client.embeddings.create(
        input=review["comment"],
        model="text-embedding-3-small",
    )
    embedding=response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professorName"],
        "metadata":{
            "review":review["comment"],
            "subject":review["course"],
            "stars":review["rating"]
        }
    })


In [22]:
pinecone = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pinecone.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ms1"
)

{'upserted_count': 20}

In [ ]:
index.describe_index_stats()